In [1]:
import scipy
import scipy.sparse
import numpy
import os
import re
import multiprocessing
import subprocess
import pickle
import matplotlib.pyplot
import time
import statistics
import sklearn


In [2]:
# using full dataset
CSR_mat_full = scipy.sparse.load_npz('./testdata/sparse_matrix_mumbach.npz')
str(CSR_mat_full)
print("total number of reads imported", CSR_mat_full.sum()/2)

total number of reads imported 180841744.0


In [3]:
##############
#thing to make it faster
##############
CSR_mat_full = CSR_mat_full[0:584662,0:584662]
print("MATRIX sliced to get only chromosome 1")
print("total number of reads in chromosome 1", CSR_mat_full.sum()/2)
print("matrix size", CSR_mat_full.shape)

MATRIX sliced to get only chromosome 1
total number of reads in chromosome 1 13647630.0
matrix size (584662, 584662)


In [4]:
#needed to load the variables
with open("./testdata/variables.pi","rb") as picklefile:
    frag_index,frag_prop,frag_amount,valid_chroms,chroms_offsets,distribution_nice_fragments = pickle.load(picklefile)


In [6]:
%matplotlib qt
matplotlib.pyplot.scatter(range(1000),distribution_nice_fragments)

In [14]:
CSR_mat_full.diagonal().sum()

1599232.0

In [13]:
%matplotlib qt
matplotlib.pyplot.plot(CSR_mat_full.diagonal())
matplotlib.pyplot.plot(CSR_mat_full.diagonal(k=1).tolist())
matplotlib.pyplot.plot(CSR_mat_full.diagonal(k=2).tolist())